In [91]:
import cv2
import math
from win32api import GetSystemMetrics
import time
import numpy as np

#enter path of the image I sent you
imagePath = 'C:\\Users\\Salil kulkarni\\Desktop\\TARQ\\ParkingApp\\customgoogle.PNG'

In [92]:
selectedPoints = []
def click_event_selectedpoints(event, x, y, flags, params):

# checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        selectedPoints.append([x,y])

def get_resized_for_display_img(img):
    screen_w, screen_h = GetSystemMetrics(0), GetSystemMetrics(1)
    #print("screen size",screen_w, screen_h)
    h,w,channel_nbr = img.shape
    # img get w of screen and adapt h
    h = h * (screen_w / w)
    w = screen_w
    if h > screen_h: #if img h still too big
        # img get h of screen and adapt w
        w = w * (screen_h / h)
        h = screen_h
    w, h = w*0.9, h*0.9 # because you don't want it to be that big, right ?
    w, h = int(w), int(h) # you need int for the cv2.resize
    return cv2.resize(img, (w, h))

#using new bgr values for the new image in low red.
def getBorderContour_text(img):

    # Upper and lower color limit customized for snazzy maps
    low_red = (55, 55, 255)
    high_yellow = (242,251,256)

    low_gray = (241,241,241)
    high_gray = (244,243,241)

    # # Upper and lower color limit customized for google maps
    # low_yellow = (240,251,255)
    # high_yellow = (240,251,255)

    # low_gray = (244,243,241)
    # high_gray = (244,243,241)

    # create masks
    yellow_mask = cv2.inRange(img, low_red, low_red )
    gray_mask = cv2.inRange(img, low_red, low_red)

    # combine masks
    combined_mask = cv2.bitwise_or(yellow_mask, gray_mask)
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_DILATE,kernel)

    blank = np.zeros(img.shape[:2], dtype='uint8')

    # findcontours
    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)
        area = cv2.contourArea(c)
        if(area>200):
            for eps in np.linspace(0.001, 0.0075, 10):
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps * peri, True)
            
            # draw the approximated contour on the image  
            cv2.drawContours(blank, [approx], -1, 255, thickness=1)
            # cv2.drawContours(img, [c], -1, 255, thickness=1)


    # cv2.imshow("image",img)
    # cv2.waitKey(0)

    return blank

In [93]:
def detectCorner(image):

    # making a copy of the image to have the original image untouched in main loop
    imageSub = image.copy()

    # convet to gray and perform Harris corner detection
    gray = cv2.cvtColor(imageSub,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)

    # result is dilated for marking the corners, not important
    dst = cv2.dilate(dst,None)

    # threshold for an optimal value, it may vary depending on the image.
    imageSub[dst>0.01*dst.max()]=[0,0,255]

    return imageSub

In [94]:
img = cv2.imread(imagePath)
img = get_resized_for_display_img(img)
image = get_resized_for_display_img(img)
# convert image to HSV
#hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

#will show black and white image of borders of the buildings. Click on this image at different points to encircle a 
#a structure for corner detection. Coordinates will not be displayed. Coordinates will be stored and resultant mask will
#be displayed.
cntr_img = getBorderContour_text(img)

cv2.imshow("cntr trace", cntr_img)
cv2.setMouseCallback('cntr trace', click_event_selectedpoints)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [95]:
low_red = (55,55,255)

red_mask = cv2.inRange(img, low_red, low_red )


# combine masks

original_frame = cv2.imread(imagePath)
original_frame = get_resized_for_display_img(original_frame)
points = np.array(selectedPoints)
blank = np.zeros(original_frame.shape[:2], dtype='uint8')
poly_pts = np.array( points ,dtype=np.int32)
poly_pts = poly_pts.reshape((-1, 1, 2))
polymask = cv2.fillPoly(blank, pts=[poly_pts],color=255)
# masked = cv2.bitwise_and(original_frame,original_frame,mask=polymask)
combined_mask = cv2.bitwise_or(red_mask, red_mask)
masked = cv2.bitwise_and(original_frame,original_frame,mask=combined_mask)


#will display masked quadrilateral whose points were selected prior
cv2.imshow("out", masked)
cv2.waitKey(0)
# cv2.destroyAllWindows()

13

In [96]:
img = cv2.imread(imagePath)
original_frame = get_resized_for_display_img(img)

operatedImage = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)

# modify the data type
# setting to 32-bit floating point
# operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) #uncomment if you feel image is noisy (not needed)
operatedImage = np.float32(operatedImage)

# apply the cv2.cornerHarris method
# to detect the corners with appropriate
# values as input parameters
#~~~~~~~~~~~~~#
#for obtaining corners for ray masking
#~~~~~~~~~~~~~#
dest = cv2.cornerHarris(operatedImage, 2, 5, 0.07)

# Results are marked through the dilated corners
dest = cv2.dilate(dest, None)

# for pts in poly_pts:
#     dest[pts[0]][pts[1]] = 0
# Reverting back to the original image,
# with optimal threshold value
size = list(original_frame.shape)

suro = np.zeros(original_frame.shape, dtype='uint8')
suro[dest > 0.01 * dest.max()]=[45, 255, 108]

suro = cv2.bitwise_and(suro,suro,mask=polymask)

cv2.imshow('Image with Borders', suro)
cv2.waitKey(0)

#     # De-allocate any associated memory 
# cv2.waitKey(0)

final = cv2.bitwise_or(original_frame, suro)

# original_frame[1] = final_green


# the window showing output image with corners
cv2.imshow('Image with Borders', final)

#     # De-allocate any associated memory 
cv2.waitKey(0)
cv2.destroyAllWindows()
